In [9]:
import os
print(os.getcwd())
os.listdir()

C:\Users\robin\code\ssi\ssi_project\logs\connect_bt


['.ipynb_checkpoints', 'Analysis.ipynb', 'devices.txt', 'teraterm.log']

In [12]:
def parse_log(file_content):
    content_split = file_content.split('\n\n')[1:]
    content_split[0] = '\n'+content_split[0]
    content_split
    all_pkt = []
    for pkt in content_split:
        pkt_split = pkt.split('.')
        meta = pkt_split[0].split('\n')
        if meta[1]!='':
            time = meta[1]
            chan = meta[2]
            rate = meta[3]
            content_pkt_split = pkt_split[1].split('\n')
            content = ''
            for el in content_pkt_split[1:]:
                content += el[2:]+' '
            new_pkt = {
                'time':time,
                'chan':chan,
                'data_rate':rate,
                'content': content,
                'devices': {}
            }
            all_pkt += [new_pkt]
    return all_pkt

In [13]:
all_pkt = parse_log(content)

In [45]:
def reverse(addr):
    addr = addr.split(' ')
    addr = addr[::-1]
    reversed_addr = ''
    for num in addr:
        reversed_addr += num + ' '
    reversed_addr = reversed_addr[:-1]
    return reversed_addr

#print(reverse(devices['Muvo'])=='f1 b0 5c 3c 02 00')
def find_devices(pkt_list, devices):
    for pkt in pkt_list:
        for name, addr in devices.items():
            if addr in pkt['content'] or reverse(addr) in pkt['content']:
                if name in pkt['devices']:
                    pkt['devices'][name] += 1
                else:
                    pkt['devices'][name] = 1
    return pkt_list

In [46]:
def count_devices(all_pkt, devices):
    count = {}
    for pkt in all_pkt:
        for name in devices.keys():
            if name in pkt['devices'].keys():
                if name in count.keys():
                    count[name] += 1
                else:
                    count[name] = 0
    return count, len(all_pkt)

In [47]:
def summarise(log_filepath, devices):
    log_file = open(log_filepath, 'r')
    content = log_file.read()
    packets = parse_log(content)
    if packets:
        packets = find_devices(packets, devices)
        return count_devices(packets, devices)
    else:
        return {}, 0

In [48]:
devices = {
    'Muvo':'00 02 3c 5c b0 f1',
    'UE boom':'88 c6 26 c7 ed b6',
    'iphone robin': 'f4 31 c3 8c f1 7d',
    'BT': 'e8 08 bf 57 8c 81'
}
summarise('teraterm.log', devices)

({'UE boom': 411, 'iphone robin': 335}, 569)